<a href="https://www.kaggle.com/code/amannnx/multimodal-health-coach?scriptVersionId=235100731" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
pip install --upgrade google-cloud-speech google-cloud google-cloud-videointelligence google-generativeai vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 73.8 MB/s eta 0:00:00
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.8.4
    Uninstalling google-generativeai-0.8.4:
      Successfully uninstalled google-generativeai-0.8.4
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.79.0
    Uninstalling google-cloud-aiplatform-1.79.0:
      Successfully uninstalled google-cloud-aiplatform-1.79.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Multimodal Health Coach - Vertex AI Implementation
# Using Speech-to-Text v2, Gemini Pro Vision, and Video Intelligence API

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests

from PIL import Image
import io
import base64
import cv2
import time
from typing import Dict, List, Tuple, Any, Optional

# Google Cloud libraries
from google.cloud.aiplatform_v1beta1.services.dataset_service import DatasetServiceClient
from google.cloud import speech, videointelligence
import google.generativeai as genai
import vertexai
from vertexai.generative_models import GenerativeModel, Part

In [4]:
# Configure APIs and authentication
def setup_google_cloud_credentials():
    try:
        # Set your Google Cloud credentials
        # For secure implementation, use environment variables or secrets management
        # os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/service-account-key.json"
        
        # Configure Gemini API
        # Replace with your actual API key
        os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"
        genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
        
        # Initialize Vertex AI
        # Replace with your actual project ID and location
        # vertexai.init(project="your-project-id", location="us-central1")
        
        print("Google Cloud credentials configured successfully.")
    except Exception as e:
        print(f"Error setting up Google Cloud credentials: {e}")

# Initialize APIs
setup_google_cloud_credentials()

# ----------------------------------------------------------
# 1. Image Understanding Module - Food Analysis
# ----------------------------------------------------------

class FoodImageAnalyzer:
    """Module for analyzing food images and extracting nutritional information."""
    
    def __init__(self):
        # Initialize Gemini Vision model for image understanding
        self.gemini_vision_model = genai.GenerativeModel('gemini-pro-vision')
    
    def analyze_image(self, image_path: str) -> Dict:
        """
        Analyze food image and return nutritional estimates.
        
        Args:
            image_path: Path to the food image
            
        Returns:
            Dictionary containing nutritional information
        """
        try:
            # Load image
            img = Image.open(image_path)
            
            # Generate image analysis prompt
            prompt = """
            Analyze this food image and provide the following information in JSON format:
            1. Food items identified in the image
            2. Estimated nutritional content for each item (calories, protein, carbs, fat)
            3. Portion size estimate
            4. Confidence score for each identification
            
            Format the response as a valid JSON object.
            """
            
            # Process with Gemini Vision
            response = self.gemini_vision_model.generate_content([prompt, img])
            
            # Extract JSON from response
            nutrition_data = self._extract_json_from_response(response.text)
            
            return nutrition_data
            
        except Exception as e:
            print(f"Error analyzing food image: {e}")
            return {"error": str(e)}
    
    def _extract_json_from_response(self, response_text: str) -> Dict:
        """Extract JSON data from model response."""
        try:
            # Find JSON content in the response
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                return json.loads(json_str)
            else:
                return {"error": "No JSON found in response"}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON in response"}

# ----------------------------------------------------------
# 2. Video Understanding Module - Exercise Form Analysis
# ----------------------------------------------------------

class ExerciseVideoAnalyzer:
    """Module for analyzing exercise videos and providing form feedback."""
    
    def __init__(self):
        # Initialize Gemini Video model (or equivalent)
        self.gemini_video_model = genai.GenerativeModel('gemini-pro-vision')
        
        # For Google Cloud Video Intelligence API
        self.video_client = videointelligence.VideoIntelligenceServiceClient()
    
    def analyze_exercise_form(self, video_path: str, exercise_type: str) -> Dict:
        """
        Analyze exercise form in video and provide feedback.
        
        Args:
            video_path: Path to the exercise video
            exercise_type: Type of exercise being performed
            
        Returns:
            Dictionary containing form analysis and feedback
        """
        try:
            # Extract frames from video
            frames = self._extract_frames(video_path)
            
            # For full implementation, use Video Intelligence API for pose estimation
            # For demonstration, we're using frame analysis with Gemini Vision
            
            prompt = f"""
            Analyze these frames from a {exercise_type} exercise video.
            Provide the following in JSON format:
            1. Overall form assessment (scale 1-10)
            2. Specific form issues identified
            3. Recommendations for improvement
            4. Risk assessment for injury
            
            Format the response as a valid JSON object.
            """
            
            # Process key frames with Gemini Vision
            sample_frame = frames[len(frames)//2]  # Take middle frame for demo
            response = self.gemini_video_model.generate_content([prompt, sample_frame])
            
            # Extract JSON from response
            analysis_data = self._extract_json_from_response(response.text)
            
            return analysis_data
            
        except Exception as e:
            print(f"Error analyzing exercise video: {e}")
            return {"error": str(e)}
    
    def analyze_with_video_intelligence(self, video_path: str) -> Dict:
        """Use Video Intelligence API for deeper video analysis."""
        try:
            with open(video_path, "rb") as video_file:
                input_content = video_file.read()
            
            # Configure the request
            features = [
                videointelligence.Feature.PERSON_DETECTION,
                videointelligence.Feature.LABEL_DETECTION
            ]
            
            # Create the request
            request = videointelligence.AnnotateVideoRequest(
                input_content=input_content,
                features=features
            )
            
            # Make the API call
            operation = self.video_client.annotate_video(request=request)
            print("Processing video. This might take some time...")
            
            # Process the response
            result = operation.result(timeout=180)
            
            # Process person detection results
            person_annotations = result.annotation_results[0].person_detection_annotations
            
            # Extract and return relevant information
            workout_analysis = []
            for person in person_annotations:
                for track in person.tracks:
                    for timestamped_object in track.timestamped_objects:
                        # Extract pose landmarks if available
                        pose_landmarks = []
                        if hasattr(timestamped_object, 'landmarks'):
                            for landmark in timestamped_object.landmarks:
                                pose_landmarks.append({
                                    'name': landmark.name,
                                    'x': landmark.point.x,
                                    'y': landmark.point.y
                                })
                        
                        # Add to analysis
                        workout_analysis.append({
                            'time': timestamped_object.time_offset.seconds + timestamped_object.time_offset.microseconds / 1000000,
                            'confidence': timestamped_object.confidence,
                            'pose_landmarks': pose_landmarks
                        })
            
            return {"pose_analysis": workout_analysis}
            
        except Exception as e:
            print(f"Error in video intelligence analysis: {e}")
            return {"error": str(e)}
    
    def _extract_frames(self, video_path: str, max_frames: int = 5) -> List[Image.Image]:
        """Extract representative frames from video."""
        frames = []
        try:
            video = cv2.VideoCapture(video_path)
            total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
            
            # Extract evenly distributed frames
            frame_indices = np.linspace(0, total_frames-1, max_frames, dtype=int)
            
            for idx in frame_indices:
                video.set(cv2.CAP_PROP_POS_FRAMES, idx)
                success, frame = video.read()
                if success:
                    # Convert BGR to RGB
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frames.append(Image.fromarray(frame_rgb))
            
            video.release()
            return frames
        except Exception as e:
            print(f"Error extracting video frames: {e}")
            return frames
    
    def _extract_json_from_response(self, response_text: str) -> Dict:
        """Extract JSON data from model response."""
        try:
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                return json.loads(json_str)
            else:
                return {"error": "No JSON found in response"}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON in response"}

# ----------------------------------------------------------
# 3. Audio Understanding Module - Health Habits Analysis
# ----------------------------------------------------------

class AudioLogAnalyzer:
    """Module for analyzing audio logs of health habits."""
    
    def __init__(self):
        # Initialize speech-to-text and language models
        self.speech_client = speech.SpeechClient()
        self.language_model = genai.GenerativeModel('gemini-pro')
    
    def analyze_audio_log(self, audio_path: str) -> Dict:
        """
        Analyze audio log of health habits.
        
        Args:
            audio_path: Path to the audio file
            
        Returns:
            Dictionary containing extracted habits and analysis
        """
        try:
            # 1. Transcribe audio to text
            transcript = self._transcribe_audio(audio_path)
            
            # 2. Extract health habits and analyze with language model
            prompt = """
            Analyze this health journal entry and provide the following in JSON format:
            1. Key health habits mentioned
            2. Sentiment analysis for each habit (positive, negative, neutral)
            3. Consistency assessment
            4. Potential areas for improvement
            5. Overall wellness assessment
            
            Health journal transcript:
            """ + transcript
            
            response = self.language_model.generate_content(prompt)
            
            # Extract JSON from response
            habits_data = self._extract_json_from_response(response.text)
            
            return habits_data
            
        except Exception as e:
            print(f"Error analyzing audio log: {e}")
            return {"error": str(e)}
    
    def _transcribe_audio(self, audio_path: str) -> str:
        """Transcribe audio file to text using Speech-to-Text v2."""
        try:
            # Load audio file
            with open(audio_path, "rb") as audio_file:
                content = audio_file.read()
            
            # Configure speech recognition
            audio = speech.RecognitionAudio(content=content)
            config = speech.RecognitionConfig(
                encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
                sample_rate_hertz=16000,
                language_code="en-US",
                enable_automatic_punctuation=True,
                model="latest_long",  # Use enhanced model for better medical terminology recognition
            )
            
            # Detect speech
            response = self.speech_client.recognize(config=config, audio=audio)
            
            # Concatenate transcripts
            transcript = ""
            for result in response.results:
                transcript += result.alternatives[0].transcript + " "
            
            return transcript.strip()
            
        except Exception as e:
            print(f"Error transcribing audio: {e}")
            # For demonstration, return dummy transcript if transcription fails
            return "I've been trying to eat more vegetables and cut down on processed foods. Yesterday I walked 8,000 steps and did a 20-minute yoga session. I've been sleeping about 6 hours per night and feeling tired in the afternoons. I've been drinking about 5 glasses of water daily."
    
    def _extract_json_from_response(self, response_text: str) -> Dict:
        """Extract JSON data from model response."""
        try:
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                return json.loads(json_str)
            else:
                return {"error": "No JSON found in response"}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON in response"}

# ----------------------------------------------------------
# 4. Recommendation Engine - Personalized Health Guidance
# ----------------------------------------------------------

class HealthRecommendationEngine:
    """Engine for generating personalized health recommendations."""
    
    def __init__(self):
        self.recommendation_model = genai.GenerativeModel('gemini-pro')
        self.user_profiles = {}  # In a real app, this would be a database
    
    def create_or_update_user_profile(self, user_id: str, profile_data: Dict) -> Dict:
        """Create or update user health profile."""
        if user_id not in self.user_profiles:
            self.user_profiles[user_id] = profile_data
        else:
            self.user_profiles[user_id].update(profile_data)
        
        return self.user_profiles[user_id]
    
    def generate_recommendations(self, user_id: str, 
                                food_data: Dict = None, 
                                exercise_data: Dict = None, 
                                habits_data: Dict = None) -> Dict:
        """
        Generate personalized health recommendations based on user profile and analysis data.
        
        Args:
            user_id: User identifier
            food_data: Food analysis data (optional)
            exercise_data: Exercise analysis data (optional)
            habits_data: Health habits data (optional)
            
        Returns:
            Dictionary containing personalized recommendations
        """
        try:
            # Get user profile
            if user_id not in self.user_profiles:
                return {"error": "User profile not found"}
            
            user_profile = self.user_profiles[user_id]
            
            # Prepare input data for recommendation generation
            input_data = {
                "user_profile": user_profile,
                "food_data": food_data if food_data else {},
                "exercise_data": exercise_data if exercise_data else {},
                "habits_data": habits_data if habits_data else {}
            }
            
            # Generate prompt for recommendations
            prompt = f"""
            Based on the following user data, generate personalized health recommendations in JSON format:
            
            USER PROFILE:
            {json.dumps(input_data['user_profile'], indent=2)}
            
            FOOD ANALYSIS:
            {json.dumps(input_data['food_data'], indent=2) if input_data['food_data'] else "No food data available"}
            
            EXERCISE ANALYSIS:
            {json.dumps(input_data['exercise_data'], indent=2) if input_data['exercise_data'] else "No exercise data available"}
            
            HEALTH HABITS:
            {json.dumps(input_data['habits_data'], indent=2) if input_data['habits_data'] else "No habits data available"}
            
            Generate the following in JSON format:
            1. Meal plan for next 3 days with specific recipes and nutritional breakdowns
            2. Exercise routine recommendations with specific exercises, sets, reps, and form tips
            3. Habit modification suggestions with specific actionable steps
            4. Overall wellness assessment and progress tracking
            
            Format the response as a valid JSON object.
            """
            
            # Generate recommendations
            response = self.recommendation_model.generate_content(prompt)
            
            # Extract JSON from response
            recommendations = self._extract_json_from_response(response.text)
            
            return recommendations
            
        except Exception as e:
            print(f"Error generating recommendations: {e}")
            return {"error": str(e)}
    
    def _extract_json_from_response(self, response_text: str) -> Dict:
        """Extract JSON data from model response."""
        try:
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                return json.loads(json_str)
            else:
                return {"error": "No JSON found in response"}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON in response"}

# ----------------------------------------------------------
# 5. Health Coach Agent - Coordinating System
# ----------------------------------------------------------

class HealthCoachAgent:
    """Coordinating agent system for the health coach."""
    
    def __init__(self):
        self.food_analyzer = FoodImageAnalyzer()
        self.exercise_analyzer = ExerciseVideoAnalyzer()
        self.audio_analyzer = AudioLogAnalyzer()
        self.recommendation_engine = HealthRecommendationEngine()
        self.conversation_model = genai.GenerativeModel('gemini-pro')
        self.memory = {}  # Simple memory system to maintain context
    
    def process_input(self, user_id: str, input_type: str, input_data: Any) -> Dict:
        """
        Process user input based on type and update user profile.
        
        Args:
            user_id: User identifier
            input_type: Type of input ('food_image', 'exercise_video', 'audio_log', 'text')
            input_data: Input data (path to file or text)
            
        Returns:
            Dictionary containing analysis results
        """
        try:
            result = {}
            
            # Process based on input type
            if input_type == 'food_image':
                result = self.food_analyzer.analyze_image(input_data)
                self._update_memory(user_id, 'food_data', result)
                
            elif input_type == 'exercise_video':
                # For video, we need exercise type - in real app, would get from context
                exercise_type = "general"  # Default
                if isinstance(input_data, tuple) and len(input_data) == 2:
                    video_path, exercise_type = input_data
                else:
                    video_path = input_data
                
                result = self.exercise_analyzer.analyze_exercise_form(video_path, exercise_type)
                self._update_memory(user_id, 'exercise_data', result)
                
            elif input_type == 'audio_log':
                result = self.audio_analyzer.analyze_audio_log(input_data)
                self._update_memory(user_id, 'habits_data', result)
                
            elif input_type == 'text':
                # Process text input to update profile or answer questions
                result = self._process_text_input(user_id, input_data)
                
            else:
                result = {"error": "Unsupported input type"}
            
            return result
            
        except Exception as e:
            print(f"Error processing input: {e}")
            return {"error": str(e)}
    
    def generate_health_plan(self, user_id: str) -> Dict:
        """Generate comprehensive health plan based on all available data."""
        try:
            # Get all data from memory
            user_memory = self.memory.get(user_id, {})
            
            # Generate recommendations
            recommendations = self.recommendation_engine.generate_recommendations(
                user_id,
                food_data=user_memory.get('food_data'),
                exercise_data=user_memory.get('exercise_data'),
                habits_data=user_memory.get('habits_data')
            )
            
            return recommendations
            
        except Exception as e:
            print(f"Error generating health plan: {e}")
            return {"error": str(e)}
    
    def _process_text_input(self, user_id: str, text_input: str) -> Dict:
        """Process textual input from user."""
        try:
            # Get user memory
            user_memory = self.memory.get(user_id, {})
            
            # Generate prompt for processing text input
            prompt = f"""
            User ID: {user_id}
            User Memory: {json.dumps(user_memory, indent=2)}
            
            User Message: {text_input}
            
            Analyze this message and provide a response in JSON format with:
            1. Intent classification (query, update_profile, request_recommendation, other)
            2. Extracted information (health goals, preferences, constraints)
            3. Response message
            4. Profile updates (if any)
            
            Format the response as a valid JSON object.
            """
            
            # Generate response
            response = self.conversation_model.generate_content(prompt)
            
            # Extract JSON from response
            result = self._extract_json_from_response(response.text)
            
            # Update profile if needed
            if 'profile_updates' in result and result['profile_updates']:
                self._update_user_profile(user_id, result['profile_updates'])
            
            return result
            
        except Exception as e:
            print(f"Error processing text input: {e}")
            return {"error": str(e)}
    
    def _update_memory(self, user_id: str, data_type: str, data: Dict) -> None:
        """Update user memory with new data."""
        if user_id not in self.memory:
            self.memory[user_id] = {}
        
        self.memory[user_id][data_type] = data
    
    def _update_user_profile(self, user_id: str, profile_updates: Dict) -> None:
        """Update user profile with new information."""
        self.recommendation_engine.create_or_update_user_profile(user_id, profile_updates)
    
    def _extract_json_from_response(self, response_text: str) -> Dict:
        """Extract JSON data from model response."""
        try:
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                return json.loads(json_str)
            else:
                return {"error": "No JSON found in response"}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON in response"}

# ----------------------------------------------------------
# 6. Demo Functions - For Testing and Visualization
# ----------------------------------------------------------

def demo_food_analysis(image_path: str):
    """Demo function to showcase food image analysis."""
    analyzer = FoodImageAnalyzer()
    result = analyzer.analyze_image(image_path)
    
    # Display results
    plt.figure(figsize=(12, 8))
    
    # Display image
    plt.subplot(1, 2, 1)
    img = Image.open(image_path)
    plt.imshow(img)
    plt.title("Food Image")
    plt.axis('off')
    
    # Display analysis
    plt.subplot(1, 2, 2)
    plt.axis('off')
    plt.text(0.1, 0.9, "Food Analysis Results", fontsize=14, fontweight='bold')
    
    # Format and display results
    y_pos = 0.8
    for food_item, details in result.get('food_items', {}).items():
        plt.text(0.1, y_pos, f"Item: {food_item}", fontsize=12)
        y_pos -= 0.05
        
        if 'calories' in details:
            plt.text(0.1, y_pos, f"Calories: {details['calories']}", fontsize=10)
            y_pos -= 0.04
        
        if 'protein' in details:
            plt.text(0.1, y_pos, f"Protein: {details['protein']}", fontsize=10)
            y_pos -= 0.04
        
        if 'carbs' in details:
            plt.text(0.1, y_pos, f"Carbs: {details['carbs']}", fontsize=10)
            y_pos -= 0.04
        
        if 'fat' in details:
            plt.text(0.1, y_pos, f"Fat: {details['fat']}", fontsize=10)
            y_pos -= 0.04
        
        if 'confidence' in details:
            plt.text(0.1, y_pos, f"Confidence: {details['confidence']}", fontsize=10)
            y_pos -= 0.06
    
    plt.tight_layout()
    plt.show()
    
    # Return the full result for further processing
    return result

def demo_full_health_coach(user_id: str = "demo_user"):
    """Demo function to showcase the complete health coach workflow."""
    # Initialize health coach agent
    coach = HealthCoachAgent()
    
    # 1. Create user profile
    profile_data = {
        "name": "Demo User",
        "age": 35,
        "height": 175,  # cm
        "weight": 70,    # kg
        "goals": ["lose weight", "improve strength", "eat healthier"],
        "dietary_restrictions": ["no red meat"],
        "activity_level": "moderate",
        "current_exercise": "2-3 times per week",
    }
    
    coach.recommendation_engine.create_or_update_user_profile(user_id, profile_data)
    print("User profile created.")
    
    # 2. Simulate food image analysis (using placeholder data for demo)
    food_data = {
        "food_items": {
            "chicken salad": {
                "calories": 350,
                "protein": "25g",
                "carbs": "15g",
                "fat": "20g",
                "confidence": 0.92
            },
            "whole grain bread": {
                "calories": 120,
                "protein": "5g",
                "carbs": "22g",
                "fat": "2g",
                "confidence": 0.85
            }
        },
        "portion_size": "medium",
        "meal_type": "lunch"
    }
    
    coach._update_memory(user_id, 'food_data', food_data)
    print("Food analysis completed.")
    
    # 3. Simulate exercise video analysis
    exercise_data = {
        "exercise_type": "squat",
        "form_score": 7.5,
        "issues": [
            "knees extending past toes",
            "back slightly rounded at bottom position"
        ],
        "recommendations": [
            "Focus on pushing knees outward during descent",
            "Maintain a more upright chest position",
            "Consider box squats to practice proper depth"
        ],
        "injury_risk": "low to moderate"
    }
    
    coach._update_memory(user_id, 'exercise_data', exercise_data)
    print("Exercise analysis completed.")
    
    # 4. Simulate habits analysis from audio log
    habits_data = {
        "habits": {
            "sleep": {
                "duration": "6 hours",
                "quality": "poor",
                "sentiment": "negative"
            },
            "hydration": {
                "intake": "5 glasses",
                "sentiment": "neutral"
            },
            "physical_activity": {
                "steps": "8000",
                "additional": "20 min yoga",
                "sentiment": "positive"
            },
            "nutrition": {
                "positive": "more vegetables",
                "negative": "processed foods",
                "sentiment": "neutral"
            }
        },
        "overall_assessment": "moderate",
        "improvement_areas": ["sleep", "hydration"]
    }
    
    coach._update_memory(user_id, 'habits_data', habits_data)
    print("Health habits analysis completed.")
    
    # 5. Generate comprehensive health plan
    health_plan = coach.generate_health_plan(user_id)
    
    # 6. Display results
    print("\n=== PERSONALIZED HEALTH PLAN ===\n")
    print(json.dumps(health_plan, indent=2))
    
    return health_plan

# ----------------------------------------------------------
# 7. Main Function - Run this for a full demonstration
# ----------------------------------------------------------

def main():
    """Main function to demonstrate the health coach functionality."""
    print("Multimodal Health Coach - Vertex AI Implementation")
    print("Using Speech-to-Text v2, Gemini Pro Vision, and Video Intelligence API")
    print("\nInitializing health coach system...\n")
    
    # Run the demo with a sample user
    health_plan = demo_full_health_coach("demo_user")
    
    print("\nDemonstration completed.")
    return health_plan

# Run the main function if this script is executed directly
if __name__ == "__main__":
    main()

Google Cloud credentials configured successfully.
Multimodal Health Coach - Vertex AI Implementation
Using Speech-to-Text v2, Gemini Pro Vision, and Video Intelligence API

Initializing health coach system...

User profile created.
Food analysis completed.
Exercise analysis completed.
Health habits analysis completed.
Error generating recommendations: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

=== PERSONALIZED HEALTH PLAN ===

{
  "error": "400 API key not valid. Please pass a valid API key. [reason: \"API_KEY_INVALID\"\ndomain: \"googleapis.com\"\nmetadata {\n  key: \"service\"\n  value: \"generativelanguage.googleapis.com\"\n}\n, locale: \"en-US\"\nmessage: \"API key not valid. Please pass a valid API key.\"\n]"
}

Demonstration completed.
